- Define a graph of computations in python => Tensorflow takes that graph and runs it using optimized C++ code.<br>
- possiblity of breaking the graph into multiple chunks or distributed computing<br>
- In addition to its main API, there are also some **simple APIs:**
    - TF.Learn (tensorflow.contrib.learn) is compatible with Scikit-Learn.
    - TF.slim (tensorflow.contrib.slim)
    - Keras
    - Pretty Tensor



# Creating and running a graph
A TF program is usu. split into 2 parts:
- construction phase
- execution phase

In [11]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2
print(f)

Tensor("add_5:0", shape=(), dtype=int32)


A TF session takes care of placing the operations onto devices such as CPUs and GPUs and running them, and it holds all the vars values.

In [12]:
session_conf=tf.ConfigProto(device_count={'GPU': 0})
sess=tf.Session(config=session_conf)

sess.run(x.initializer)
sess.run(y.initializer)

result=sess.run(f)
print(result)

sess.close()

42


Repeating `sess.run()` is sometimes cumbersome!<br>
The `with` statement is used to wrap the execution of a block with *methods* defined by a context manager.<br>
- The context manager handles the entry into, and the exit from, the desired runtime context for the execution of the block of code.

In [13]:
session_conf=tf.ConfigProto(device_count={'GPU': 0})
with tf.Session(config=session_conf) as sess: 
    x.initializer.run()
    y.initializer.run()
    
    result=f.eval()
    
    # automatically close the session.

print(result)

42


Initilizing every single variable is combersome!. Use `tf.global_variables_initializer()`.

## Example 1: using `with`

In [12]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

session_conf=tf.ConfigProto(device_count={'GPU': 0})
init=tf.global_variables_initializer()
with tf.Session(config=session_conf) as sess:
    init.run()
    
    result=f.eval()
    
print(result)

42


## Example 2:

In [11]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

session_conf=tf.ConfigProto(device_count={'GPU': 0})
sess=tf.Session(config=session_conf)

init=tf.global_variables_initializer()

sess.run(init)
result=sess.run(f)
print(result)

sess.close()

42


## Interactive session

In [17]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

init = tf.global_variables_initializer()

sess = tf.InteractiveSession()

init.run()

result = f.eval()

sess.close()

print(result)

42


c:\users\ashkan\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\client\session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


# Lifecycle of a node
- A node is automatically added to the default graph.
- **start**: initilizer is run; **end**: the session is closed.
- TF does not reuse the result of the previous evaluations. In the following example, TF evaluates w & x twice.

In [13]:
import tensorflow as tf

# Running the same commands multiple times may result in many duplicate nodes. restart the jupyter 
# kernel or use the following command:
tf.reset_default_graph()

w=tf.constant(3)
x=w+2
y=x+5
z=x*3

session_conf=tf.ConfigProto(device_count={'GPU': 0})
with tf.Session(config=session_conf) as sess:
    print(y.eval())
    print(z.eval())

10
15


# Feeding data to the training algorithm

In [14]:
import tensorflow as tf

tf.reset_default_graph()

input_data=[[4, 5, 6],
            [7, 8, 9]]

A=tf.constant(input_data)

B = A + 5

session_conf=tf.ConfigProto(device_count={'GPU': 0})
with tf.Session(config=session_conf) as sess:
    result=B.eval()

print(result)

[[ 9 10 11]
 [12 13 14]]


We need a way to replace new data at every iteration. **Place holder** nodes just output the data you tell them to output at runtime.<br>

In [15]:
import tensorflow as tf

tf.reset_default_graph()

chunk=[]
chunk.append([[4, 5, 6],
                   [7, 8, 9]])
chunk.append([[1, 1, 1],
                   [2, 2, 2]])

A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

result=[]
session_conf=tf.ConfigProto(device_count={'GPU': 0})
with tf.Session(config=session_conf) as sess:
    for i in range(0,2):
        result.append(B.eval(feed_dict={A: chunk[i]}))

print(result[0])
print(result[1])

[[ 9. 10. 11.]
 [12. 13. 14.]]
[[6. 6. 6.]
 [7. 7. 7.]]
